In [1]:
import pandas as pd
import numpy as np
import yfinance
from stock_indicators import indicators
from stock_indicators import Quote
from lightweight_charts import Chart
import asyncio
import nest_asyncio
nest_asyncio.apply()


In [2]:
df = yfinance.download("AAPL", start="2023-01-01", end="2024-01-01", multi_level_index=False)
# Reset the index to make the 'Date' column available
df.reset_index(inplace=True)
# Rename columns to match the expected format
df.rename(columns={
    'Date': 'date',
    'Open': 'open',
    'High': 'high',
    'Low': 'low',
    'Close': 'close',
    'Volume': 'volume'
}, inplace=True)

quotes = [
    Quote(d, o, h, l, c, v)
    for d, o, h, l, c, v in zip(
        df['date'], df['open'], df['high'], df['low'], df['close'], df['volume']
    )
]

df['rsi'] = [r.rsi for r in indicators.get_rsi(quotes, 14)]

df['dynamic'] = [d.dynamic for d in indicators.get_dynamic(quotes, 14)]

df['SMA_short'] = [ind.sma for ind in indicators.get_sma(quotes, 12)]
df['SMA_long'] = [ind.sma for ind in indicators.get_sma(quotes, 26)]    

# Generate buy signals
df['Buy_Signal'] = np.where((df['SMA_short'].shift(1) < df['SMA_long'].shift(1)) &
                                (df['SMA_short'] > df['SMA_long']),
                                df['close'], np.nan)

# Generate sell signals
df['Sell_Signal'] = np.where((df['SMA_short'].shift(1) > df['SMA_long'].shift(1)) &
                                 (df['SMA_short'] < df['SMA_long']),
                                 df['close'], np.nan)

df.tail()

C:\Users\jwang33\AppData\Local\Temp\ipykernel_23932\212508018.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yfinance.download("AAPL", start="2023-01-01", end="2024-01-01", multi_level_index=False)
[*********************100%***********************]  1 of 1 completed


,date,close,high,low,open,volume,rsi,dynamic,SMA_short,SMA_long,Buy_Signal,Sell_Signal
245,2023-12-22,191.974686,193.769488,191.349970,193.541408,37149600,54.672805,192.101690,193.978549,191.308773,NaN,NaN
246,2023-12-26,191.429306,192.262250,191.211152,191.984602,28919300,53.090054,192.020514,193.877738,191.436157,NaN,NaN
247,2023-12-27,191.528458,191.875525,189.485754,190.874010,48087700,53.354439,191.961331,193.666196,191.568117,NaN,NaN
248,2023-12-28,191.954865,193.025800,191.548303,192.510161,34049900,54.541030,191.960561,193.699252,191.649354,NaN,NaN
249,2023-12-29,190.913666,192.767962,190.120379,192.272159,42672100,51.121338,191.833175,193.519109,191.721437,NaN,NaN


In [3]:
if __name__ == '__main__':
    chart = Chart(inner_width=1, inner_height=0.8)
    chart.time_scale(visible=False)
             
    Mcginley = chart.create_line('dynamic', color='orange', width=2, )
    df1 = df[['date', 'dynamic']]
    Mcginley.set(df1)

    chart2 = chart.create_subchart(width=1, height=0.2, sync=True)
    line = chart2.create_line('rsi', color='blue', width=2)
    df2 = df[['date', 'rsi']]
    
    
    chart.set(df)
    line.set(df2)

    chart.show(block=True)


In [4]:
# Assuming 'chart' is your Lightweight Chart object and 'candlestick_series' is your candlestick series
for index, row in df.iterrows():
        if not pd.isna(row['Buy_Signal']):
            chart.add_series_marker(
                series=candlestick_series,
                time=index.timestamp(),  # Convert datetime to timestamp
                position='belowBar',
                color='green',
                shape='arrowUp',
                text='Buy'
            )
        if not pd.isna(row['Sell_Signal']):
            chart.add_series_marker(
                series=candlestick_series,
                time=index.timestamp(),
                position='aboveBar',
                color='red',
                shape='arrowDown',
                text='Sell'
            )

AttributeError: 'Chart' object has no attribute 'add_series_marker'